# VGG implementation in TensorFlow Slim

[VGGNet](https://arxiv.org/pdf/1409.1556.pdf) to classify flowers into the 17 categories of the Oxford Flowers data set.

This notebook is inspired by the notebooks freely available from [TensorFlow-LiveLessons](https://github.com/the-deep-learners/TensorFlow-LiveLessons) and from [TensorFlow-Examples](https://github.com/aymericdamien/TensorFlow-Examples).

## Preliminary steps

### Global params

In [ ]:
epochs = 10
batch_size = 32
test_size = 0.1
pool_size = 2
n_maxpool_layers = 5
padding = 'VALID'
learning_rate = 1e-4
dropout = 0.5
display_steps = 2

### Set seed

In [ ]:
import numpy as np
np.random.seed(42)

### Load dependencies

In [ ]:
import tensorflow as tf
import tensorflow.contrib.slim as slim
import tensorflow.contrib.layers as layers
from sklearn.model_selection import train_test_split
import math

### Load the data

In [ ]:
import tflearn.datasets.oxflower17 as oxflower17
X, Y = oxflower17.load_data(one_hot=True)

In [ ]:
num_examples, img_height, img_width, img_channels = X.shape

X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=test_size, random_state=42)

In [ ]:
print("Shape of training set {}".format(X_train.shape))

In [ ]:
print("Shape of labels {}".format(Y_train.shape))

In [ ]:
n_classes = Y.shape[1]
print(n_classes)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
fig, axes = plt.subplots(2,2)
axes[0,0].imshow(X_train[0])
axes[0,1].imshow(X_train[1])
axes[1,0].imshow(X_train[2])
axes[1,1].imshow(X_train[3])

In [ ]:
print(Y_train[0:4])

## Define the network architecture

### Placeholders for images and targets

In [ ]:
x = tf.placeholder(tf.float32, shape=[None, img_height, img_width, img_channels])
y_ = tf.placeholder(tf.float32, shape=[None, n_classes])
keep_prob = tf.placeholder(tf.float32) # dropout (keep probability)

### Network

In [ ]:
# Parameters

# Conv 1
k_conv1 = 3
n_conv1 = 64

# Conv 2
k_conv2 = 3
n_conv2 = 128

# Conv 3
k_conv3 = 3
n_conv3 = 256

# Conv 4
k_conv4 = 3
n_conv4 = 512

# Conv 5
k_conv5 = 3
n_conv5 = 512

# FC 6
n_fc6 = 4096

# FC 7
n_fc7 = 4096

In [ ]:
def vgg16(x, debug, batch_norm=True):
    with slim.arg_scope([layers.conv2d, layers.fully_connected],
                       activation_fn=tf.nn.relu,
                       weights_initializer=tf.contrib.layers.xavier_initializer()):
        # convolution 1
        net = slim.repeat(x, 2, layers.conv2d, n_conv1, [k_conv1, k_conv1],
                          padding=padding, scope='conv1')
        net = layers.max_pool2d(net, [pool_size, pool_size],
                              padding=padding, scope='pool1')
        if batch_norm:
            net = layers.batch_norm(net)
        if debug:
            print('Conv1 shape {}'.format(net.shape))
            
        # convolution 2
        net = slim.repeat(net, 2, layers.conv2d, n_conv2, [k_conv2, k_conv2],
                          padding=padding, scope='conv2')
        net = layers.max_pool2d(net, [pool_size, pool_size],
                              padding=padding, scope='pool2')
        if batch_norm:
            net = layers.batch_norm(net)
        if debug:
            print('Conv2 shape {}'.format(net.shape))

        # convolution 3
        net = slim.repeat(net, 3, layers.conv2d, n_conv3, [k_conv3, k_conv3],
                          padding=padding, scope='conv3')
        net = layers.max_pool2d(net, [pool_size, pool_size],
                              padding=padding, scope='pool3')
        if batch_norm:
            net = layers.batch_norm(net)
        if debug:
            print('Conv3 shape {}'.format(net.shape))

        # convolution 4
        net = slim.repeat(net, 3, layers.conv2d, n_conv4, [k_conv4, k_conv4],
                          padding=padding, scope='conv4')
        net = layers.max_pool2d(net, [pool_size, pool_size],
                              padding=padding, scope='pool4')
        if batch_norm:
            net = layers.batch_norm(net)
        if debug:
            print('Conv4 shape {}'.format(net.shape))
            
        # convolution 5
        net = slim.repeat(net, 3, layers.conv2d, n_conv5, [k_conv5, k_conv5],
                          padding=padding, scope='conv5')
        net = layers.max_pool2d(net, [pool_size, pool_size],
                              padding=padding, scope='pool5')
        if batch_norm:
            net = layers.batch_norm(net)
        if debug:
            print('Conv5 shape {}'.format(net.shape))
            
        # tf.summary.histogram('pool_5', net)
        
        # fully connected layers
        
        net = layers.flatten(net)
        net = layers.fully_connected(net, n_fc6, scope='fc6')
        net = tf.nn.dropout(net, keep_prob, name='dropout6')
        net = layers.fully_connected(net, n_fc7, scope='fc7')
        net = tf.nn.dropout(net, keep_prob, name='dropout7')
        net = layers.fully_connected(net, n_classes, activation_fn=None, scope='fc8')
        
        # tf.summary.histogram('scores', net)

    return net

### Filters

In [ ]:
# Compute dense layer size
if padding == 'SAME':
    downsampling_factor = math.pow(pool_size, n_maxpool_layers)
    dense_layer_size = int(img_height / downsampling_factor) * int(img_width / downsampling_factor) * n_conv5
else:
    dense_layer_size = n_conv5

## Run the architecture

### Build the model

In [ ]:
predictions = vgg16(x, True, True )
predictions.shape

### Define loss and optimization method

In [ ]:
tf.losses.softmax_cross_entropy(y_, predictions)
total_loss = tf.losses.get_total_loss()
tf.summary.scalar('loss', total_loss)

#optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
optimizer = tf.train.AdamOptimizer(learning_rate)
train_op = optimizer.minimize(total_loss)

### Evaluation

In [ ]:
accuracy, update_op_acc = tf.metrics.accuracy(y_, predictions)
tf.summary.scalar('accuracy', accuracy)

### Trainining

In [ ]:
with tf.Session() as sess:
    # Merge all the summaries and write them out to /tmp/mnist_logs (by default)
    merged = tf.summary.merge_all()
    train_writer = tf.summary.FileWriter('log/VGG/run1',sess.graph)

    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
    n_batches = int(X_train.shape[0] / batch_size)
    idx = np.arange(n_batches)
    print("N. of batches {}".format(n_batches))
    steps = 0
    for epoch in range(epochs):
        sum_loss = 0.0
        # loop over batches with shuffling
        np.random.shuffle(idx)
        for i in range(n_batches):
            batch_x = X_train[idx[i]*batch_size: (idx[i]+1)*batch_size]
            batch_y = Y_train[idx[i]*batch_size: (idx[i]+1)*batch_size]
            
            sess.run([train_op, update_op_acc],feed_dict={x: batch_x, y_: batch_y, keep_prob: 1.0-dropout} )
            batch_loss, summary = sess.run([total_loss, merged],
                                            feed_dict={x: batch_x, y_: batch_y, keep_prob: 1.0})
            accuracy.eval()
            if steps % display_steps == 0:
                train_writer.add_summary(summary, steps)
            sum_loss += batch_loss 
            steps += 1
            print('.', sep='', end='')
        print('')
        print("Epoch = {} / {}; loss = {:.3f}; accuracy = {:.2f}".format(epoch+1, epochs,
                                                                         sum_loss/n_batches,
                                                                         accuracy.eval()))
    print("Training Complete. Testing Model.\n")
    
    test_loss = loss.eval({x: X_test[:batch_size], y_: Y_test[:batch_size], keep_prob: 1.0})
    test_acc = accuracy.eval({x: X_test[:batch_size], y_: Y_test[:batch_size], keep_prob: 1.0})
    print("Test Cost:", '{:.3f}'.format(test_loss))
    print("Test Accuracy: ", '{:.2f}'.format(test_acc), "%", sep='')